# Groundhog plotting basics

The geotechnical Python library ``groundhog`` (https://github.com/snakesonabrain/groundhog) is a general-purpose toolkit for geotechnical engineering. As communication about geotechnical test data and parameter profiles is one of the key tasks for geotechnical engineers, ``groundhog`` has a number of plotting routines which make this task easier. Built on top of the open-source Plotly library (https://plotly.com/python/), these plots are interactive and easy to create with just a few lines of code.

In this article, three basic plotting types are introduced:

   - Cumulative grain size plot
   - Plasticity chart
   - Mini-log soil profile plot

## Cumulative grain size plot

Plotting the result of sieving or hydrometer analysis tells the engineer a lot about the composition of a soil sample. In ``groundhog``, this is achieved with the ``PSDChart`` class which provides a plotting template for granulometric results with indication of the grain size classes according to the ISO 14688-1 standard.

We can first import the class from ``groundhog``.

In [ ]:
from groundhog.siteinvestigation.labtesting.indextests import PSDChart

Next, we generate some grain sizes (in mm) with corresponding % passing values:

In [ ]:
d = [2.500, 2.000, 1.250, 1.000, 0.630, 0.500, 0.315, 0.250, 0.160, 0.125, 0.080, 0.063]
pct_passing = [99.9, 99.8, 98.4, 96.9, 90.8, 83.8, 62.7, 48.8, 26.8, 14.0, 4.0, 2.9]

Creating the plot is just a matter of creating a ``PSDChart`` object (called ``psdplot`` here) and adding the data using the ``add_trace`` method. You can add as many traces as you want by repeating the ``.add_trace`` method.

In [ ]:
psdplot = PSDChart(plot_title="Example PSD chart")
psdplot.add_trace(grainsize=d, pctpassing=pct_passing, name="Soil 1", showlegend=True)
psdplot.show()

## Plasticity chart

A plasticity chart summarises the results from Atterberg limit tests and show how a soil can be classified in terms of its plasticity.

A plasticity chart can easily be created in ``groundhog`` using the ``PlasticityChart`` class. We can import this class first:

In [ ]:
from groundhog.siteinvestigation.labtesting.indextests import PlasticityChart

We can then create a number of generic plasticity test results. The liquid limit (water content at which the soil starts to behave as a liquid) and the plasticity index (the water content range in which the soil behaves plastically) are defined:

In [ ]:
liquidlimits = [40, 70, 60]
plastictyindices = [10, 40, 30]

The syntax for creating the plasticity chart is again straightforward. An object of the ``PlasticityChart`` class is first created and the the data is added using the ``.add_trace`` method.

In [ ]:
plasticityplot = PlasticityChart(plot_title="Example plasticity chart")
plasticityplot.add_trace(ll=liquidlimits, pi=plastictyindices, name="Campaign 1")
plasticityplot.show()

The chart shows two lines. The A-line is an empirical line separating the clays and the silts with clays plotting above the A-line and silts below. The second line is called the U-line and is an upper bound for plasticity measurements.

The abbreviations shown in the chart are the Unified Soil Classification System (USCS) acronyms. Their verbose version is also included in ``groundhog``. A function which gives the verbose name for a given abbreviation (``uscs_categories``) can be imported. The Python dictionary linking the acronyms with the verbose descriptions (``USCS_DICTIONARY``) can also be used.

In [ ]:
from groundhog.siteinvestigation.classification.categories import uscs_categories, USCS_DICTIONARY

In [ ]:
USCS_DICTIONARY

In [ ]:
uscs_categories('ML')

## Mini-log soil profile plot

Whenever in-situ or laboratory test results are plotted against depth, it is instructive to also display the soil stratigraphy. We can first import some CPT data and a definition of the stratigraphy from an Excel file. The example data is from the Borssele offshore wind farm (https://offshorewind.rvo.nl/). The data is used under a Creative Commons license.

In [ ]:
import pandas as pd

The CPT data is found in the tab ``'Data'``:

In [ ]:
cpt_df = pd.read_excel('Data/Seabed CPT excel export.xlsx', sheet_name='Data')
cpt_df.head()

The definition of the stratigraphy is found in the tab ``'Layering'``. The layering can be imported as a ``groundhog`` ``SoilProfile`` object. This inherits from the Pandas dataframe but offers additional functionality for dealing with geotechnical profiles. To import data from Excel into a ``SoilProfile`` object, the ``read_excel`` method for ``groundhog`` ``SoilProfile`` objects needs to be imported:

In [ ]:
from groundhog.general.soilprofile import read_excel

The layering has four columns, one for the top and one for the bottom of each layer, a descriptor of the soil type and a column with the total (bulk) unit weight.

In [ ]:
layering = read_excel('Data/Seabed CPT excel export.xlsx', sheet_name='Layering')
layering

``SoilProfile`` objects contain the necessary stratigraphical information for plotting. The ``LogPlot`` objects creates plots with one or more panels from such ``SoilProfile`` objects and allows the user to add data to those plots. First the ``LogPlot`` can be imported.

In [ ]:
from groundhog.general.plotting import LogPlot

For those using JupyterLab, Plotly plots may sometimes render with an awkward heigth. I hope the JupyterLab people fix this soon, bit in the meantime, the code below fixes things.

In [ ]:
import plotly.io as pio
pio.templates['plotly'].layout['autosize'] = False
for key in pio.templates.keys():
    pio.templates[key].layout['autosize'] = False

Next, we need to define which color to connect to which soil type (from the ``'Soil type'`` column). We can create a Python dictionary for this mapping:

In [ ]:
colordict = {
    'SAND': 'yellow',
    'CLAY': 'brown',
    'SILT': 'wheat'
}

We can now create a plot showing the cone tip resistance, sleeve friction and pore pressure at the shoulder next to the stratigraphic log. We start out by declaring which soil profile we want to create the ``LogPlot``, then specifying the number and panels and adding the color mapping.

In [ ]:
cptplot = LogPlot(soilprofile=layering, no_panels=3, fillcolordict=colordict)

To add data to the plot. The ``.add_trace`` method is used. The ``x`` and ``z`` arguments are arrays (or Pandas dataframe columns) of equal length which give the values and depths to be plotted. A name can be given to each trace and the ``showlegend`` keyword defines whether the name needs to be shown in the legend or not. The keyword argument ``panel_no`` defines in which panel the plotted trace is shown.

In [ ]:
cptplot.add_trace(x=cpt_df['qc [MPa]'], z=cpt_df['z [m]'], name='qc', showlegend=False, panel_no=1)
cptplot.add_trace(x=cpt_df['fs [MPa]'], z=cpt_df['z [m]'], name='fs', showlegend=False, panel_no=2)
cptplot.add_trace(x=cpt_df['u2 [MPa]'], z=cpt_df['z [m]'], name='u2', showlegend=False, panel_no=3)

At this point, we are still preparing the plot for display. We now have to add the titles, ranges and tick intervals of the axes. For the x-axes, this needs to be set for each panel. The z-axis only needs to be set once as this axis is shared between all panels.

In [ ]:
cptplot.set_xaxis(title='qc [MPa]', panel_no=1, range=(0, 100), dtick=10)
cptplot.set_xaxis(title='fs [MPa]', panel_no=2, range=(0, 1), dtick=0.1)
cptplot.set_xaxis(title='u2 [MPa]', panel_no=3, range=(-0.5, 2.5), dtick=0.5)
cptplot.set_zaxis(title='z [m]')

Finally, the ``.show()`` method displays the plot. The position of the deepest clay layer is clearly confirmed by the CPT results. For the shallowest clay layer, the CPT data suggests a thinner layer. Such differences may arise because of offsets between CPT tests and sample boreholes.

In [ ]:
cptplot.show()

When parameters are defined in the ``SoilProfile`` object (as is the case for the total unit weight from the ``layering`` soil profile), we can add those to the plot using another method. We will plot the cone tip resistance and the total (bulk) unit weight in a second chart. First, we can add the cone tip resistance trace. The example below also shows how the line can be formatted using the standard Plotly syntax.

In [ ]:
qc_uw_plot = LogPlot(soilprofile=layering, no_panels=2, fillcolordict=colordict)
qc_uw_plot.add_trace(x=cpt_df['qc [MPa]'], z=cpt_df['z [m]'], name='qc', showlegend=False, panel_no=1, line=dict(color='black'))

To now add data from a ``SoilProfile`` object, we can use the method ``.add_soilparameter_trace`` and specify the parameter name (column header) from the ``SoilProfile`` object. Note that when parameters vary linearly, the parameter name is the column header without ``from`` or ``to``.

In [ ]:
qc_uw_plot.add_soilparameter_trace(
    parametername='Total unit weight [kN/m3]', panel_no=2, legendname='Bulk unit weight', showlegend=False,
    line=dict(color='red', dash='dot'))

We can then again set the axes. Here, we restrict the depth axis to a range where there is CPT data.

In [ ]:
qc_uw_plot.set_xaxis(title='qc [MPa]', panel_no=1, range=(0, 100), dtick=10)
qc_uw_plot.set_xaxis(title='Bulk unit weight [kN/m3]', panel_no=2, range=(13, 22), dtick=1)
qc_uw_plot.set_zaxis(title='z [m]', range=(35, 0))

Finally the plot is displayed. The specified bulk unit weight is nicely displayed alongside the cone tip resistance.

In [ ]:
qc_uw_plot.show()

## Conclusions

The ``groundhog`` library has a number of useful plotting routines which create instructive plots allowing interpretation of geotechnical data. If you have any suggestions for plots to include in ``groundhog`` don't hesitate to reach out.